In [0]:
from PIL import Image
import os
import numpy as np
import torch
import torch.utils.data
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
%%shell

# Install pycocotools
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI
python setup.py build_ext install

In [0]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [0]:
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    return model

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params    = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:03<00:00, 47.7MB/s]


In [0]:
singan_dir = '/content/gdrive/My Drive/singan/SinGAN'
models_dir = os.path.join(singan_dir, 'models')
images_dir = os.path.join(singan_dir, 'Input/People')
masks_dir  = os.path.join(singan_dir, 'Input/Masks')
assert os.path.exists(singan_dir)
assert os.path.exists(models_dir)
assert os.path.exists(images_dir)
assert os.path.exists(masks_dir)

model_name = 'fastrcnn_segmentation_model_12012019-184816.pth'
model_path = os.path.join(models_dir, model_name)
assert os.path.exists(model_path)

image_name = 'field_hockey1.jpg'
image_path = os.path.join(images_dir, image_name)
assert os.path.exists(image_path)

In [0]:
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [0]:
import transforms as T
image = Image.open(image_path).convert("RGB")
image, _ = T.ToTensor()(image, None)

with torch.no_grad(): 
  prediction = model([image.to(device)])

In [0]:
prediction

In [0]:
Image.fromarray(image.mul(255).permute(1, 2, 0).byte().numpy())

In [0]:
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())

In [0]:
bin_treshold = 0.01
ones  = torch.ones_like(prediction[0]['masks'])
zeros = torch.zeros_like(prediction[0]['masks'])  
masks = torch.where(prediction[0]['masks'] < bin_treshold, ones, zeros)

In [103]:
masks_names = [image_name[:-4]+'_mask_{}.jpg'.format(i) for i in range(prediction[0]['masks'].shape[0])]
masks_names

['field_hockey1_mask_0.jpg',
 'field_hockey1_mask_1.jpg',
 'field_hockey1_mask_2.jpg',
 'field_hockey1_mask_3.jpg',
 'field_hockey1_mask_4.jpg',
 'field_hockey1_mask_5.jpg',
 'field_hockey1_mask_6.jpg']

In [0]:
for i, mask_name in enumerate(masks_names):
  image_t = Image.fromarray(masks[i, :, :, :].squeeze().mul(255).byte().cpu().numpy())
  image_t = image_t.save(mask_name)

In [105]:
import shutil
for mask_name in masks_names:
  mask_name_dest = os.path.join(masks_dir, mask_name)
  print('Save {} to drive... '.format(mask_name), end=' ')
  shutil.copyfile(mask_name, mask_name_dest)
  print('Done.')

Save field_hockey1_mask_0.jpg to drive...  Done.
Save field_hockey1_mask_1.jpg to drive...  Done.
Save field_hockey1_mask_2.jpg to drive...  Done.
Save field_hockey1_mask_3.jpg to drive...  Done.
Save field_hockey1_mask_4.jpg to drive...  Done.
Save field_hockey1_mask_5.jpg to drive...  Done.
Save field_hockey1_mask_6.jpg to drive...  Done.


In [106]:
from google.colab import files
for mask_name in masks_names:
  print('Download {} to Local... '.format(mask_name), end=' ')
  files.download(mask_name)
  print('Done.')

Download field_hockey1_mask_0.jpg to Local...  Done.
Download field_hockey1_mask_1.jpg to Local...  Done.
Download field_hockey1_mask_2.jpg to Local...  Done.
Download field_hockey1_mask_3.jpg to Local...  Done.
Download field_hockey1_mask_4.jpg to Local...  Done.
Download field_hockey1_mask_5.jpg to Local...  Done.
Download field_hockey1_mask_6.jpg to Local...  Done.


In [107]:
masked_names = [image_name[:-4]+'_masked_{}.jpg'.format(i) for i in range(masks.shape[0])]
masked_names

['field_hockey1_masked_0.jpg',
 'field_hockey1_masked_1.jpg',
 'field_hockey1_masked_2.jpg',
 'field_hockey1_masked_3.jpg',
 'field_hockey1_masked_4.jpg',
 'field_hockey1_masked_5.jpg',
 'field_hockey1_masked_6.jpg']

In [0]:
for i, masked_name in enumerate(masked_names):
  masked = torch.mul(image.to(device),masks[i,:,:,:])
  masked = Image.fromarray(masked.mul(255).permute(1, 2, 0).byte().cpu().numpy())
  masked = masked.save(masked_name)

In [109]:
for masked_name in masked_names:
  masked_name_dest = os.path.join(masks_dir, masked_name)
  print('Save {} to drive... '.format(masked_name), end=' ')
  shutil.copyfile(masked_name, masked_name_dest)
  print('Done.')

Save field_hockey1_masked_0.jpg to drive...  Done.
Save field_hockey1_masked_1.jpg to drive...  Done.
Save field_hockey1_masked_2.jpg to drive...  Done.
Save field_hockey1_masked_3.jpg to drive...  Done.
Save field_hockey1_masked_4.jpg to drive...  Done.
Save field_hockey1_masked_5.jpg to drive...  Done.
Save field_hockey1_masked_6.jpg to drive...  Done.


In [110]:
from google.colab import files
for masked_name in masked_names:
  print('Download {} to Local... '.format(masked_name), end=' ')
  files.download(masked_name)
  print('Done.')

Download field_hockey1_masked_0.jpg to Local...  Done.
Download field_hockey1_masked_1.jpg to Local...  Done.
Download field_hockey1_masked_2.jpg to Local...  Done.
Download field_hockey1_masked_3.jpg to Local...  Done.
Download field_hockey1_masked_4.jpg to Local...  Done.
Download field_hockey1_masked_5.jpg to Local...  Done.
Download field_hockey1_masked_6.jpg to Local...  Done.
